In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle
import numpy as np
import os
from tqdm.notebook import tqdm
from skimage.color import rgb2gray
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
import matplotlib.image as mpimg

In [ ]:
# load files
with open('/content/drive/MyDrive/deeplearning_file/sg/imageswithoutcut.pickle', 'rb') as f:
    images = pickle.load(f)

with open('/content/drive/MyDrive/deeplearning_file/sg/labelswithoutcut.pickle', 'rb') as f:
    labels = pickle.load(f)

In [ ]:
# 카테고리별 경로로 이미지 파일 불러오기
canpath = '/content/drive/MyDrive/deeplearning_file/recyclable_materials/labeling_Img_1/can/cut/'
glasspath = '/content/drive/MyDrive/deeplearning_file/recyclable_materials/labeling_Img_1/glass/cut/'
paperpath = '/content/drive/MyDrive/deeplearning_file/recyclable_materials/labeling_Img_1/paper/cut/'
plasticpath = '/content/drive/MyDrive/deeplearning_file/recyclable_materials/labeling_Img_1/plastic/cut/'
canfiles = os.listdir(canpath)
glassfiles = os.listdir(glasspath)
paperfiles =  os.listdir(paperpath)
plasticfiles = os.listdir(plasticpath)

In [ ]:
folders = [canpath, glasspath,  paperpath,  plasticpath]
files = [ canfiles,  glassfiles,  paperfiles,  plasticfiles]

checks = set()
for i in range(4) :
    path = folders[i]
    for file in files[i]:
        image = mpimg.imread(path + file)
        checks.add(image.shape[2])
print(checks)

In [ ]:
checks = set()
for i in range(4) :
    path = folders[i]
    for file in files[i]:
        image = mpimg.imread(path + file)
        if image.shape[2] == 4:
            for _ in range(len(set(image[:,:,3].reshape(-1)))):
                checks.add(set(image[:,:,3].reshape(-1)).pop())
print(checks)

In [ ]:
# 이미지 리사이징 & array 변환 
for i in range(4) :
    path = folders[i]
    bar_total = tqdm(files[i])

    for file in bar_total:
        image = mpimg.imread(path + file)
        if image.shape[2] == 4:
            image = image[:,:,:3]
        images = np.append(images, np.array([resize(image, (128, 128, 3))]), axis=0)
        if i == 0 :
            labels.append('can')
        elif i == 1 :
            labels.append('glass')
        elif i == 2 :
            labels.append('paper')
        elif i == 3 :
            labels.append('plastic')
        else:
            print(f'{i}는 존재하지 않습니다.')

In [ ]:
# 라벨인코딩
encoder = LabelEncoder()
encoder.fit(labels)
labels_encoded = encoder.transform(labels)
labels_encoded[:3], encoder.classes_

In [ ]:
# 데이터셋 train, test 나누기
X_train = np.concatenate((images[:int(s0*0.8)], images[s0:s0+int(s1*0.8)], images[s0+s1:s0+s1+int(s2*0.8)], images[s0+s1+s2:s0+s1+s2+int(s3 *0.8)], images[s0+s1+s2+s3:] ), axis=0)
X_test = np.concatenate((images[int(s0*0.8):s0], images[s0+int(s1*0.8):s0+s1], images[s0+s1+int(s2*0.8):s0+s1+s2], images[s0+s1+s2+int(s3*0.8):s0+s1+s2+s3]), axis=0)
y_train = np.concatenate((labels_encoded[:int(s0*0.8)], labels_encoded[s0:s0+int(s1*0.8)], labels_encoded[s0+s1:s0+s1+int(s2*0.8)], labels_encoded[s0+s1+s2:s0+s1+s2+int(s3 *0.8)], labels_encoded[s0+s1+s2+s3:]), axis=0)
y_test = np.concatenate((labels_encoded[int(s0*0.8):s0], labels_encoded[s0+int(s1*0.8):s0+s1], labels_encoded[s0+s1+int(s2*0.8):s0+s1+s2], labels_encoded[s0+s1+s2+int(s3*0.8):s0+s1+s2+s3]), axis=0)

X_train.shape, X_test.shape

In [ ]:
#### 모델 ####

In [ ]:
import time
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Input
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Dropout,Dense,Conv2D,MaxPooling2D,Flatten
from functools import partial
import  matplotlib.pyplot as plt


In [ ]:
# 바닐라 CNN


model = models.Sequential([
    layers.Conv2D(64, (5, 5), activation='relu', input_shape=(128,128,3)),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (5, 5), activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (5, 5), activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(4, activation='softmax')
    
])

model.summary()

In [ ]:
# resnet 50


# # The identity block
# def identity_block(X, f, filters, stage, block):
#   conv_name_base = 'res' + str(stage) + block + '_branch'
#   bn_name_base = 'bn' + str(stage) + block + '_branch'
#   F1, F2, F3 = filters
#   X_shortcut = X
#   # first step of main path
#   X = tf.keras.layers.Conv2D(filters=F1, kernel_size=1, strides=1, padding='valid', name=conv_name_base + '2a',
#   kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
#   X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
#   X = tf.keras.layers.Activation('relu')(X)
#   # second step of main path
#   X = tf.keras.layers.Conv2D(filters=F2, kernel_size=f, strides=1, padding='same', name=conv_name_base + '2b',
#   kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
#   X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
#   X = tf.keras.layers.Activation('relu')(X)
#   # third step of main path
#   X = tf.keras.layers.Conv2D(filters=F3, kernel_size=1, strides=1, padding='valid', name=conv_name_base + '2c',
#   kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
#   X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2c')(X)
#   # add shortcut value and pass it through a ReLU activation
#   X = tf.keras.layers.Add()([X, X_shortcut])
#   X = tf.keras.layers.Activation('relu')(X)
#   return X

# # Convolutional Block
# def convolutional_block(X, f, filters, stage, block, s=2):
#   conv_name_base = 'res'+str(stage)+block+'_branch'
#   bn_name_base = 'bn'+str(stage)+block+'_branch'
#   F1, F2, F3 = filters
#   X_shortcut = X
#   # first step of main path
#   X = tf.keras.layers.Conv2D(filters=F1, kernel_size=1, strides=s, padding='valid', name=conv_name_base+'2a',
#   kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
#   X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2a')(X)
#   X = tf.keras.layers.Activation('relu')(X)
#   # second step of main path
#   X = tf.keras.layers.Conv2D(filters=F2, kernel_size=f, strides=1, padding='same', name=conv_name_base+'2b',
#   kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
#   X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2b')(X)
#   X = tf.keras.layers.Activation('relu')(X)
#   # third step of main path
#   X = tf.keras.layers.Conv2D(filters=F3, kernel_size=1, strides=1, padding='valid', name=conv_name_base+'2c',
#   kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
#   X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'2c')(X)
#   # shortcut path
#   X_shortcut = tf.keras.layers.Conv2D(filters=F3, kernel_size=1, strides=s, padding='valid', name=conv_name_base+'1',
#   kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X_shortcut)
#   X_shortcut = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base+'1')(X_shortcut)
#   # Add and pass it through a ReLU activation
#   X = tf.keras.layers.Add()([X, X_shortcut])
#   X = tf.keras.layers.Activation('relu')(X)
#   return X

In [ ]:
# def ResNet50(input_shape=(128,128,3), classes=4):
#   X_input = tf.keras.layers.Input(input_shape)
#   # zero padding
#   X = tf.keras.layers.ZeroPadding2D((3,3))(X_input)
#   # stage 1
#   X = tf.keras.layers.Conv2D(filters=64, kernel_size=7, strides=2, name='conv1',
#   kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
#   X = tf.keras.layers.BatchNormalization(axis=3, name='bn_conv1')(X)
#   X = tf.keras.layers.Activation('relu')(X)
#   X = tf.keras.layers.MaxPooling2D((3,3), strides=(2,2))(X)
#   # stage 2
#   X = convolutional_block(X, f=3, filters=[64,64,256], stage=2, block='a', s=1)
#   X = identity_block(X, 3, [64,64,256], stage=2, block='b')
#   X = identity_block(X, 3, [64,64,256], stage=2, block='c')
#   # stage 3
#   X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
#   X = identity_block(X, 3, [128, 128, 512], stage = 3, block='b')
#   X = identity_block(X, 3, [128, 128, 512], stage = 3, block='c')
#   X = identity_block(X, 3, [128, 128, 512], stage = 3, block='d')
#   # Stage 4
#   X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
#   X = identity_block(X, 3, [256, 256, 1024], stage = 4, block='b')
#   X = identity_block(X, 3, [256, 256, 1024], stage = 4, block='c')
#   X = identity_block(X, 3, [256, 256, 1024], stage = 4, block='d')
#   X = identity_block(X, 3, [256, 256, 1024], stage = 4, block='e')
#   X = identity_block(X, 3, [256, 256, 1024], stage = 4, block='f')
#   # Stage 5
#   X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
#   X = identity_block(X, 3, [512, 512, 2048], stage = 5, block='b')
#   X = identity_block(X, 3, [512, 512, 2048], stage = 5, block='c')
#   # AVGPOOL
#   X = tf.keras.layers.AveragePooling2D()(X)
#   # output layer
#   X = tf.keras.layers.Flatten()(X)
#   X = tf.keras.layers.Dense(classes, activation='softmax', name='fc'+str(classes),
#   kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
#   # Create Model
#   model = tf.keras.models.Model(inputs=X_input, outputs=X, name='ResNet50')
#   return model

# from keras import optimizers
# from tensorflow import keras

# model = ResNet50(input_shape = (128, 128, 3), classes = 4)
# adam = optimizers.Adam(learning_rate=0.001)
# model.compile(optimizer=adam,
#              loss='sparse_categorical_crossentropy',
#              metrics=['accuracy'])
# model.summary()

In [ ]:
# vgg16


# def VGG16_model():
#   tf.random.set_seed(2)
#   model = tf.keras.models.Sequential([
#   # Conv 1
#   layers.Conv2D(64, (3, 3), strides=1, padding='same', activation='relu',
#   input_shape=(128,128,3), name='conv1_1'),
#   layers.Conv2D(64, (3, 3), strides=1, padding='same', activation='relu', name='conv1_2'),
#   layers.MaxPool2D((2,2), padding='same', name='conv1_MaxPool'),
#   # Conv 2
#   layers.Conv2D(128, (3, 3), strides=1, padding='same', activation='relu', name='conv2_1'),
#   layers.Conv2D(128, (3, 3), strides=1, padding='same', activation='relu', name='conv2_2'),
#   layers.MaxPool2D((2,2), padding='same', name='conv2_MaxPool'),
#   # Conv3
#   layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu', name='conv3_1'),
#   layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu', name='conv3_2'),
#   layers.Conv2D(256, (3, 3), strides=1, padding='same', activation='relu', name='conv3_3'),
#   layers.MaxPool2D((2,2), padding='same', name='conv3_MaxPool'),
#   # Conv4
#   layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv4_1'),
#   layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv4_2'),
#   layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv4_3'),
#   layers.MaxPool2D((2,2), padding='same', name='conv4_MaxPool'),
#   # Conv5
#   layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv5_1'),
#   layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv5_2'),
#   layers.Conv2D(512, (3, 3), strides=1, padding='same', activation='relu', name='conv5_3'),
#   layers.MaxPool2D((2,2), padding='same', name='conv5_MaxPool'),
#   # Flatten
#   layers.Flatten(),
#   # Dropout
#   layers.Dropout(0.5),
#   # FC1
#   layers.Dense(512, activation='relu'),
#   # output
#   layers.Dense(4, activation='softmax'),
#   ])

#   return model
# model = VGG16_model()
# model.summary()

In [ ]:
# google-net

# def inception(x,
#               filters_1x1,
#               filters_3x3_reduce,
#               filters_3x3,
#               filters_5x5_reduce,
#               filters_5x5,
#               filters_pool):
#   path1 = layers.Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)

#   path2 = layers.Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
#   path2 = layers.Conv2D(filters_3x3, (1, 1), padding='same', activation='relu')(path2)

#   path3 = layers.Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
#   path3 = layers.Conv2D(filters_5x5, (1, 1), padding='same', activation='relu')(path3)

#   path4 = layers.MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
#   path4 = layers.Conv2D(filters_pool, (1, 1), padding='same', activation='relu')(path4)

#   return tf.concat([path1, path2, path3, path4], axis=3)

In [ ]:
# inp = layers.Input(shape=(128, 128, 3))
# input_tensor = layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=X_train.shape[1:])(inp)

# x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(input_tensor)
# x = layers.MaxPooling2D(3, strides=2)(x)

# x = layers.Conv2D(64, 1, strides=1, padding='same', activation='relu')(x)
# x = layers.Conv2D(192, 3, strides=1, padding='same', activation='relu')(x)

# x = layers.MaxPooling2D(3, strides=2)(x)

# x = inception(x,
#               filters_1x1=64,
#               filters_3x3_reduce=96,
#               filters_3x3=128,
#               filters_5x5_reduce=16,
#               filters_5x5=32,
#               filters_pool=32)

# x = inception(x,
#               filters_1x1=128,
#               filters_3x3_reduce=128,
#               filters_3x3=192,
#               filters_5x5_reduce=32,
#               filters_5x5=96,
#               filters_pool=64)

# x = layers.MaxPooling2D(3, strides=2)(x)

# x = inception(x,
#               filters_1x1=192,
#               filters_3x3_reduce=96,
#               filters_3x3=208,
#               filters_5x5_reduce=16,
#               filters_5x5=48,
#               filters_pool=64)

# aux1 = layers.AveragePooling2D((5, 5), strides=3)(x)
# aux1 = layers.Conv2D(128, 1, padding='same', activation='relu')(aux1)
# aux1 = layers.Flatten()(aux1)
# aux1 = layers.Dense(1024, activation='relu')(aux1)
# aux1 = layers.Dropout(0.7)(aux1)
# aux1 = layers.Dense(4, activation='softmax')(aux1)

# x = inception(x,
#               filters_1x1=160,
#               filters_3x3_reduce=112,
#               filters_3x3=224,
#               filters_5x5_reduce=24,
#               filters_5x5=64,
#               filters_pool=64)

# x = inception(x,
#               filters_1x1=128,
#               filters_3x3_reduce=128,
#               filters_3x3=256,
#               filters_5x5_reduce=24,
#               filters_5x5=64,
#               filters_pool=64)

# x = inception(x,
#               filters_1x1=112,
#               filters_3x3_reduce=144,
#               filters_3x3=288,
#               filters_5x5_reduce=32,
#               filters_5x5=64,
#               filters_pool=64)

# aux2 = layers.AveragePooling2D((5, 5), strides=3)(x)
# aux2 = layers.Conv2D(128, 1, padding='same', activation='relu')(aux2)
# aux2 = layers.Flatten()(aux2)
# aux2 = layers.Dense(1024, activation='relu')(aux2)
# aux2 = layers.Dropout(0.7)(aux2)
# aux2 = layers.Dense(4, activation='softmax')(aux2)

# x = inception(x,
#               filters_1x1=256,
#               filters_3x3_reduce=160,
#               filters_3x3=320,
#               filters_5x5_reduce=32,
#               filters_5x5=128,
#               filters_pool=128)

# x = layers.MaxPooling2D(3, strides=2)(x)

# x = inception(x,
#               filters_1x1=256,
#               filters_3x3_reduce=160,
#               filters_3x3=320,
#               filters_5x5_reduce=32,
#               filters_5x5=128,
#               filters_pool=128)

# x = inception(x,
#               filters_1x1=384,
#               filters_3x3_reduce=192,
#               filters_3x3=384,
#               filters_5x5_reduce=48,
#               filters_5x5=128,
#               filters_pool=128)

# x = layers.GlobalAveragePooling2D()(x)

# x = layers.Dropout(0.4)(x)
# out = layers.Dense(4, activation='softmax')(x)

# model = tf.keras.models.Model(inputs = inp, outputs = [out, aux1, aux2])
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1, mode='min', baseline=None, restore_best_weights=False)
# mc = keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
# model.compile(optimizer='adam', 
#               loss=['sparse_categorical_crossentropy','sparse_categorical_crossentropy','sparse_categorical_crossentropy'], 
#               loss_weights=[1, 0.3, 0.3], metrics=['accuracy'], 
#               )
# model.summary()

In [ ]:
# 모델 구성
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
# 모델 훈련시키기

callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=5),
             keras.callbacks.ModelCheckpoint(filepath='best_model.h5',
                                             monitor='val_loss',
                                             save_best_only=True)]
hist = model.fit(X_train, y_train,
                batch_size=32,
                epochs=100,
                validation_data = (X_test, y_test),
               callbacks=callbacks)

In [ ]:
# imagenet 사전학습 모델
# X-ception

# tf.keras.backend.clear_session()
# tf.random.set_seed(42)
# np.random.seed(42)

# xception_model = keras.applications.xception.Xception(weights="imagenet",
#                                                       include_top=False,
#                                                       input_shape = (128,128,3))
# avg = keras.layers.GlobalAveragePooling2D()(xception_model.output)
# output = keras.layers.Dense(4, activation="softmax")(avg)
# model = keras.models.Model(inputs=xception_model.input, outputs=output)

# optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
# model.compile(optimizer =optimizer ,loss = 'sparse_categorical_crossentropy',metrics =['accuracy'])

In [ ]:
# early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience = 10,restore_best_weights = True)
# lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.7,patience = 2)
# model_checkpoint =tf.keras.callbacks.ModelCheckpoint('GarbageClassifier_Xce.h5', save_best_only=True)

# class CustomCallBack(tf.keras.callbacks.Callback):
#         def on_epoch_end(self,epoch,logs={}):
#             if(logs.get('val_loss')<0.5):
#                 print("\n cancelling training!")
#                 self.model.stop_training = True
                
# mycallback = CustomCallBack()

# hist = model.fit(
#     X_train, y_train, batch_size=32,
#     epochs = 1000,
#     validation_data = (X_test, y_test),
#     callbacks= [early_stopping_cb,model_checkpoint,lr_scheduler,mycallback],
#     verbose=1)

In [ ]:
# # inception-v3

# tf.keras.backend.clear_session()
# tf.random.set_seed(42)
# np.random.seed(42)

# inception_model = keras.applications.InceptionV3(weights="imagenet",
#                                                       include_top=False,
#                                                       input_shape = (128,128,3))
# avg = keras.layers.GlobalAveragePooling2D()(inception_model.output)
# output = keras.layers.Dense(4, activation="softmax")(avg)
# model = keras.models.Model(inputs=inception_model.input, outputs=output)

# optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
# model.compile(optimizer =optimizer ,loss = 'sparse_categorical_crossentropy',metrics =['accuracy'])

In [ ]:
# epochs = 50                


# hist = model.fit(
#     X_train, y_train, batch_size=32,
#     epochs = epochs,
#     validation_data = (X_test, y_test),
#     verbose=1)

In [ ]:
# history = pd.DataFrame([range(1, epochs+1), hist.history['val_loss'], hist.history['val_accuracy'],['Inception-v3' for _ in range(epochs) ]]).T

In [ ]:
# history.columns = ['epochs', 'loss', 'accuracy', 'model', ]

In [ ]:
# # top_score = val_loss=0.385, val_acc=0.881
# history.sort_values('loss',ascending=True) 

In [ ]:
# # Efficient-net

# from keras.layers import Dense
# from keras.optimizers import Adam

# efficient_net = EfficientNetB7(
#     weights='imagenet',
#     input_shape=(128,128,3),
#     include_top=False,
#     pooling='max'
# )

# model = Sequential()
# model.add(efficient_net)
# model.add(Dense(units = 120, activation='relu'))
# model.add(Dense(units = 120, activation = 'relu'))
# model.add(Dense(units = 1, activation='softmax'))
# model.summary()

In [ ]:
# model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# epochs = 50                


# hist = model.fit(
#     X_train, y_train, batch_size=32,
#     epochs = epochs,
#     steps_per_epoch = 15,
#     validation_data = (X_test, y_test),
#     validation_steps = 7,
#     verbose=1)

In [ ]:
# 모델 평가 확인
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test loss:', test_loss, 'test_accuracy:',test_acc)

In [ ]:
# 그래프

plot_target = ['loss', 'val_loss', 'accuracy', 'val_accuracy']
plt.figure(figsize=(12, 8))

for each in plot_target:
    plt.plot(hist.history[each], label=each)

plt.legend()
plt.ylim(0,1)
plt.grid()
plt.show()

In [ ]:
# 그래프2
fig, axs = plt.subplots(2,1, figsize=(6,10))

axs[0].plot(hist.history['loss'])
axs[0].plot(hist.history['val_loss'])
axs[0].title.set_text('Loss vs Val_Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train','Val'])

axs[1].plot(hist.history['dense_4_accuracy'])
axs[1].plot(hist.history['val_dense_4_accuracy'])
axs[1].title.set_text('Accuracy vs Val_Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train', 'Val'])

In [ ]:
# Confusion Matrix
predicted_result = model.predict(X_test)
predicted_labels = np.argmax(predicted_result, axis=1)
commands = ['can',  'glass', 'paper', 'plastic']

confusion_mtx = tf.math.confusion_matrix(y_test, predicted_labels)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, xticklabels=commands, yticklabels=commands, annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()